# Unified customer feedback

The idea is to use a single map-reduce pattern for multiple different sources of customer feedback:

- Reddit
- App stores
- (Maybe) formal reviews

We'll use the extract-organize-abstract pattern with map-reduce to handle extract-organize.


In [1]:
from core import Seed, init

init()

from stored_config import Company

target = Company.find_first("98point6").to_core_company()
target

Seed(company='98point6', deprecated_product='98point6', domain='98point6.com', keywords=None, deprecated_require_news_backlinks=False, deprecated_require_reddit_backlinks=False, primary_product=None, feature_flags=FeatureFlags(require_news_backlinks=False, require_reddit_backlinks=False))

In [ ]:
# The unified version that's been drastically refactored
import data_sources.customer_experience
import data_sources.general_search
import data_sources.reddit.search

general_search_results = data_sources.general_search.search_web(target)

app_store_urls = data_sources.customer_experience.extract_app_store_urls(general_search_results)
reddit_urls = [
    result.link
    for result in data_sources.reddit.search.find_submissions(
        target, num_results=10
    )
]
customer_experience_result = data_sources.customer_experience.run(
    target, reddit_urls=reddit_urls, **app_store_urls
)

print(customer_experience_result.output_text)

In [6]:
from core import make_experiment_dir

dir = make_experiment_dir(target)

with open(f"{dir}/customer_experience.md", "w") as f:
    f.write(customer_experience_result.output_text)